In [71]:
from transformers import pipeline # importing hugging face
from nltk import sent_tokenize # sentence tokenizer, separate text to multiple sentences
import nltk
import torch
from glob import glob # for file paths
import pandas as pd # for tabular dataset

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alvee\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

### (1) LOAD THEME-CLASSIFIER MODEL

In [12]:
model_name = "facebook/bart-large-mnli"
device = 0 if torch.cuda.is_available() else 'cpu' # if gpu present use gpu, 0 is gpu number 1
# check if gpu or cpu. we get 0 so gpu
print (f"gpu->0 else 'cpu', we get : {device}")

gpu->0 else 'cpu', we get : 0


In [17]:
# LOAD THE THEME-CLASSIFIER MODEL 
# but its not recommended to read model_name from global variable, this fucntion should be inside a class

def load_model(device):
    theme_classifier = pipeline(
        "zero-shot-classification",
        model=model_name,
        device=device
    )
    return theme_classifier


In [18]:
theme_classifier = load_model(device) # install the model in puts it into gpu-mem?
# should unload it as well after work done? search how to unload


C:\Users\alvee\AppData\Roaming\Python\Python310\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [37]:
theme_list = ["friendship", "hope", "sacrifice", "battle", "self development", "betrayal", "love", "dialogue"]

In [39]:
# to classify anything with the zero-shot classifier just call it
# write any text and run it, it gives probability score for each class
theme_classifier(
    "I gave him a right hook then a left jab ",
    theme_list, 
    multi_labels=True
)




{'sequence': 'I gave him a right hook then a left jab ',
 'labels': ['battle',
  'self development',
  'sacrifice',
  'dialogue',
  'hope',
  'love',
  'betrayal',
  'friendship'],
 'scores': [0.47511664032936096,
  0.16913151741027832,
  0.10377558320760727,
  0.10101887583732605,
  0.0704294890165329,
  0.030161136761307716,
  0.029164331033825874,
  0.021202413365244865]}

### (2) LOAD DATASET

In [ ]:
# load subtitles dataset
files = glob('../data/Subtitles/*.ass') # * means get all file with .ass 
files[:5] # test , print first 5 

['../data/Subtitles\\Naruto Season 1 - 01.ass',
 '../data/Subtitles\\Naruto Season 1 - 02.ass',
 '../data/Subtitles\\Naruto Season 1 - 03.ass',
 '../data/Subtitles\\Naruto Season 1 - 04.ass',
 '../data/Subtitles\\Naruto Season 1 - 05.ass']

In [49]:
with open(files[0], 'r') as file:
    lines = file.readlines()
    
    # data cleaning
    lines = lines[27:] # data from line 27, because before it are meta data
    lines = [",".join(line.split(',')[9:]) for line in lines ]# remove everything before the 9th comma , we just want the text
    lines = [line.replace('\\N',' ') for line in lines ]# remove \\n from the text


#  check 
lines[:2]

['A long time ago, a powerful demon fox appeared with nine tails.\n',
 'With its powerful tails,\n']

In [ ]:
# combine different sentences into one-big paragraph
" ".join(lines[:10]) # joins first 10 lines
# this is the paragraph to be fed in theme-clf FNN, it can ake max 512 tokens input
# so divide it in batches [:10] / [:20] etc

"A long time ago, a powerful demon fox appeared with nine tails.\n With its powerful tails,\n it could smash mountains and create tidal waves.\n A band of Ninjas rose to defend their village from attack.\n We have to wait until the Fourth Hokage gets here!\n We can't let it get any closer to our village!\n One great Ninja was able to imprison the monster,\n but died in the process.\n This Ninja was known asâ€¦ the Fourth Hokage.\n Naruto!\n"

In [ ]:
# extract episode number from subitle file name



1

In [ ]:
# put all above from (2) Load Dataset in a single function : 
def load_subtitles_dataset(dataset_path):
    subtitles_paths = glob(dataset_path + '/*.ass')
    scripts = []
    episode_num = []
    for path in subtitles_paths: #path is the directory/file.ass
        # print(f"debug path: {path}")
        # from each path/ subtitle file read lines
        with open(path, 'r', encoding='utf-8') as file: # without  encoding='utf-8' gives error, not shown in video
            lines = file.readlines()
            
            # data/line cleaning
            lines = lines[27:] # data from line 27, because before it are meta data
            lines = [",".join(line.split(',')[9:]) for line in lines ]# remove everything before the 9th comma , we just want the text
        
        # data/line cleaning
        lines = [line.replace('\\N',' ') for line in lines ]# remove \\n from the text
        script = " ".join(lines) # all texts to one big paragraph

        episode = int(path.split('-')[-1].split('.')[0].strip())
        scripts.append(script)
        episode_num.append(episode)

    df = pd.DataFrame.from_dict({"episode" : episode_num, "script": scripts})
    return df



In [92]:
dataset_path = "../data/Subtitles" 
df = load_subtitles_dataset(dataset_path)

df.head()
    

,episode,script
0,1,"A long time ago, a powerful demon fox appeared..."
1,2,"C'mon!\n Running like a fugitive,\n Being chas..."
2,3,"C'mon!\n Running like a fugitive,\n Being chas..."
3,4,"C'mon!\n Running like a fugitive,\n Being chas..."
4,5,"C'mon!\n Running like a fugitive,\n Being chas..."


(2.1) RUN MODEL

In [ ]:
# load 'script' of the first episode [0]
script = df.iloc[0]['script']

# print script
script # one long script

# divide sciprt for tokenizer
script_sentences = sent_tokenize(script)

# print
script_sentences[:3] # display first 3 sentences

['A long time ago, a powerful demon fox appeared with nine tails.',
 'With its powerful tails,\n it could smash mountains and create tidal waves.',
 'A band of Ninjas rose to defend their village from attack.']

In [ ]:
# batch sentences
sentence_batch_size = 20
script_batches = []
for index in range(0, len(script_sentences), sentence_batch_size): # each indx jump by setence_batch_size which is 20
    sent = " ".join(script_sentences[index:index + sentence_batch_size])
    script_batches.append(sent)

# 20 sentences batched together 
script_batches[:2]

'No. Darn, this is going to be bad…\n There\'s no need to worry anymore. Lord Hokage! He\'ll be back soon. Sensei, how much longer? OK, you may open your eyes now. Congratulations…on your graduation. In celebration, we\'ll have ramen tonight! Iruka Sensei! That hurts! Naruto…\n I was going to lecture to you...\n that the road gets more difficult now that you\'re a Ninja. But I guess I\'ll just wait to tell you that until we get to the ramen stand…\n W-What do you want, you little shrimp? Quit following me! You\'re smaller than me and\n you\'re saying that you\'re gonna become the Fifth Hokage? I don\'t care if you are the 3rd Hokage\'s grandson or not. It\'s not that easy to be a Hokage! If you want it that bad, you\'re gonna have to beat me first! Next episode:  "My Name Is Konohamaru!" Watch my outstanding performance!'